Model.ipynb 分支出來分析縣市異質

In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from config import select_group
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']

In [ ]:
dataA1 = pd.read_csv('../ComputedData/Accident/DataA1_with_MYP.csv')
dataA2 = pd.read_csv('../ComputedData/Accident/DataA2_with_MYP.csv')

filtered_A2 = dataA2[dataA2['當事者順位'] == 1]
filtered_A1 = dataA1[dataA1['當事者順位'] == 1]

filtered_A1['source'] = 'A1'
filtered_A2['source'] = 'A2'
filtered_A1['num_accidents'] = 1 
filtered_A2['num_accidents'] = 1
combined_data = pd.concat([filtered_A1, filtered_A2], ignore_index=True)

In [ ]:
from shapely import wkt
import geopandas as gpd
import ast

taiwan = gpd.read_file('../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp')
taiwan = taiwan[(~taiwan['TOWNNAME'].isin(['旗津區', '頭城鎮', '蘭嶼鄉', '綠島鄉', '琉球鄉'])) & 
                (~taiwan['COUNTYNAME'].isin(['金門縣', '連江縣', '澎湖縣']))]

TM2 = 3826
hex_grid_raw = pd.read_csv('../ComputedData/Grid/hex_grid.csv')
hex_grid_raw['geometry'] = hex_grid_raw['geometry'].apply(wkt.loads)
hex_grid = gpd.GeoDataFrame(hex_grid_raw, geometry='geometry').set_crs(TM2, allow_override=True)

grid_gi_df = pd.read_csv('../ComputedData/Grid/grid_gi.csv')
grid_gi_df['accident_indices'] = grid_gi_df['accident_indices'].apply(ast.literal_eval)
grid_gi_df['geometry'] = grid_gi_df['geometry'].apply(wkt.loads)
grid_gi  = gpd.GeoDataFrame(grid_gi_df, geometry='geometry').set_crs(TM2, allow_override=True)

taiwan_tm2 = taiwan.to_crs(TM2)

taiwan_cnty = taiwan_tm2[['COUNTYNAME','geometry']].dissolve(by='COUNTYNAME')
taiwan_cnty['geometry'] = taiwan_cnty.buffer(0)
taiwan_cnty = taiwan_cnty.reset_index()

pts = hex_grid.copy()
pts['geometry'] = pts.geometry.centroid

county_join = gpd.sjoin(
    pts[['geometry']], taiwan_cnty, how='left', predicate='within'
)[['COUNTYNAME']]

print('NaN ratio:', county_join['COUNTYNAME'].isna().mean())
grid_gi['COUNTYNAME'] = county_join['COUNTYNAME']
county_join.head()

In [ ]:
def process_speed(input_data):

    bins_speed = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, float('inf')]
    labels_speed = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-110', '110+']

    input_data['速限-第1當事者'] = pd.cut(input_data['速限-第1當事者'], bins=bins_speed, labels=labels_speed, right=False).astype(str)

    return input_data

In [103]:
# 無事故的資料沒有參考性，所以要去掉
grid_filter = grid_gi[grid_gi['accident_indices'].str.len() > 0].reset_index(drop=True)
all_features_df = pd.read_csv("../ComputedData/ForModel/all_features.csv")

grid_filter['hotspot'] = grid_filter['hotspot'].apply(lambda x: 'Hotspot' if 'Hotspot' in str(x) else 'Not Hotspot')
grid_filter = grid_filter[~grid_filter['COUNTYNAME'].isna()]

In [104]:
countylst = list(grid_filter['COUNTYNAME'].unique())

county_df = {}
for county in countylst:

    filter_hotspot = grid_filter[grid_filter['hotspot'] == 'Hotspot']
    filter_not_hotspot = grid_filter[grid_filter['hotspot'] == 'Not Hotspot']

    idx_hot = filter_hotspot.loc[filter_hotspot['COUNTYNAME'] == county, 'accident_indices']
    idx_nothot = filter_not_hotspot.loc[filter_not_hotspot['COUNTYNAME'] == county, 'accident_indices']

    # 攤平
    hot_flat = [i for sublist in idx_hot for i in sublist]
    nothot_flat = [i for sublist in idx_nothot for i in sublist]

    county_df[county] = pd.concat([
        combined_data.loc[hot_flat].assign(hotspot="Hotspot"),
        combined_data.loc[nothot_flat].assign(hotspot="Not Hotspot")
    ])
    county_df[county].loc[hot_flat]['hotspot'] = 'Hotspot'
    county_df[county].loc[nothot_flat]['hotspot'] = 'Not Hotspot'


# 檢查重複
all_indices = [i for sublist in grid_filter['accident_indices'] for i in sublist]
print("總索引數量:", len(all_indices))
print("去重後數量:", len(set(all_indices)))

總索引數量: 315909
去重後數量: 315909


Odd ratio 需要嘗試